In [72]:
import openseespy.opensees as ops
import opsvis as opsv
import os
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display

In [73]:
if not os.path.exists('Data'): 
    os.mkdir('Data')

ops.wipe()  # Clear opensees model

In [74]:
# Defining base units for the simulation.
m = 1.0  # Meter (base unit for length)
s = 1.0  # Second (base unit for time)
kg = 1.0  # Kilogram (base unit for mass)

# Derived units for force and pressure.
N = kg * m / s ** 2  # Newton (unit of force)
Pa = N / m ** 2  # Pascal (unit of pressure)

# Additional conversions for different units commonly used in structural engineering.
inches = 0.0254 * m  # Conversion factor for inches to meters.
ft = 12 * inches  # Conversion factor for feet to meters.
kip = 4448.2216152548 * N  # Conversion factor for kips (1000 pounds-force) to Newtons.
ksi = 6.895 * 10 ** 6 * Pa  # Conversion factor for ksi (1000 psi) to Pascals.

In [75]:
# # Create an IntSlider widget
# slider = widgets.IntSlider(value=0, min=0, max=100, step=1, description='Column Height')

# # Function to update the variable based on slider value
# def update_variable(change):
#     global colL
#     colL = change['new']
#     # print(f'Updated colL: {colL}')

# # Observe changes in the slider's value
# slider.observe(update_variable, names='value')

# # Display the slider
# display(slider)

# # description='Column Length:'

### Inputs

In [76]:
colL = widgets.FloatText(
    value=10,
    description='Column Height (m):',
    disabled=False,
    style={'description_width': 'initial'}
)

# Cross-sectional dimensions
b = widgets.FloatText(
    value=1,
    description='Column breadth (m):',
    disabled=False,
    style={'description_width': 'initial'}
) # Breadth

d = widgets.FloatText(
    value=1,
    description='Column depth (m):',
    disabled=False,
    style={'description_width': 'initial'}
) # Depth

E = widgets.FloatText(
    value=200 * 10 ** 9,
    description='Elastic modulus (Pa):',
    disabled=False,
    style={'description_width': 'initial'}
)

Py = widgets.FloatText(
    value=-1000.0,
    description='Vertical load (N):',
    disabled=False,
    style={'description_width': 'initial'}
)

disp = widgets.FloatText(
    value=0,
    description='Horizontal displacement (m):',
    disabled=False,
    style={'description_width': 'initial'}
)

analysis = widgets.Dropdown(
    options=['Push Over', 'Earthquake'],
    value='Push Over',
    description='Analysis:',
    disabled=False,
)

output_analysis = widgets.Output(layout={'border': '1px solid black'})

# https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Layout.html#the-layout-attribute
layout = widgets.Layout(
    width = '40%',
    border='2px solid red',
    )

out_inputs_1 = widgets.Output(layout={'border': '1px solid black'})
out_inputs_2 = widgets.Output(layout={'border': '1px solid black'})

button = widgets.Button(description="Run Analysis", disabled = True, layout = layout)



@out_inputs_1.capture(clear_output = True, wait = True)
def model_definition_general(change):

    button.disabled = False

    # Nodal coordinates
    n1 = (0.0, 0.0) # Use floating point values
    n2 = (0.0, colL.value)

    # Cross sectional area
    A = b.value * d.value # Area

    # Moment of Inertia
    Iz = (b.value * d.value ** 3) / 12


    g = 9.81 # Gravitational acceleration (N / kg)

    # Compute nodal mass
    mass_x = abs(Py.value) / g # kg
    # print(f'The nodal mass value corresponding to the horizontal DOF is {mass_x:.0f} kg') # 
    massValues = [mass_x, 1 * 10 ** -9, 0.0] # ndf nodal mass values corresponding to each DOF


    # Model Definition

    ops.wipe()  # Clear opensees model

    ops.model('basic', '-ndm', 2, '-ndf', 3)    # 2D model with 3 degrees of freedom per node.

    # Create nodes
    nodal_crds = (n1, n2)
    for nodeTag, crds in enumerate(nodal_crds, start = 1): # https://docs.python.org/3/library/functions.html#enumerate
        ops.node(nodeTag, *crds)

    # Boundary Conditions - single point constraints
    constrValues = [1, 1, 1]
    ops.fix(1, *constrValues)   # Node 1 is a fixed connection
    # ops.fix(2, 0, 0, 0)         # Node 2 is free

    # Define geometric transformation: performs a linear geometric transformation of beam stiffness and resisting force from the local-coordinate system to the global-coordinate system
    ops.geomTransf('Linear', 1)     #  associate a tag of 1 to transformation

    # connectivity: (make A very large, 10e6 times its actual value)
    # element elasticBeamColumn eleTag iNode jNode A E Iz transfTag
    ops.element('elasticBeamColumn', 1, 1, 2, A * 10 ** 6, E.value, Iz, 1)     # element elasticBeamColumn 1 1 2 3600000000 4227 1080000 1;

    # nodal masses: Time - 23:30, 32:20
    ops.mass(2, *massValues)     #  node ,  Mass=Weight/g.

   
    plot_opsv()
    plot_cross_section(change)
    
    
@out_inputs_1.capture(clear_output = True, wait = True)
def plot_opsv():
    opsv.plot_model()
    plt.title('Model')
    plt.ylabel('Height')
    plt.show()

@out_inputs_2.capture(clear_output = True, wait = True)
def plot_cross_section(change):

    fig, ax = plt.subplots()
    ax.set_aspect(aspect = 'equal', adjustable = 'box')
    ax.plot([0, 0, b.value, b.value, 0], [0, d.value, d.value, 0, 0])
    ax.set_title('Cross-section')
    ax.set_xlabel('Metres')
    ax.set_ylabel('Metres')
    # xTickData = ax.get_xticks()
    ax.margins(1.0) # https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.margins.html#matplotlib.axes.Axes.margins
    ax.grid()
    plt.show()

colL.observe(model_definition_general, names='value')
b.observe(model_definition_general, names='value')
d.observe(model_definition_general, names='value')
b.observe(plot_cross_section, names = 'value')
d.observe(plot_cross_section, names='value')
E.observe(model_definition_general, names='value')
Py.observe(model_definition_general, names='value')
disp.observe(model_definition_general, names='value')
analysis.observe(model_definition_general, names='value')
HBox = widgets.HBox([out_inputs_1, out_inputs_2])

display(analysis, colL, b, d, E, Py, disp, button, HBox, output_analysis)

Dropdown(description='Analysis:', options=('Push Over', 'Earthquake'), value='Push Over')

FloatText(value=10.0, description='Column Height (m):', style=DescriptionStyle(description_width='initial'))

FloatText(value=1.0, description='Column breadth (m):', style=DescriptionStyle(description_width='initial'))

FloatText(value=1.0, description='Column depth (m):', style=DescriptionStyle(description_width='initial'))

FloatText(value=200000000000.0, description='Elastic modulus (Pa):', style=DescriptionStyle(description_width=…

FloatText(value=-1000.0, description='Vertical load (N):', style=DescriptionStyle(description_width='initial')…

FloatText(value=0.0, description='Horizontal displacement (m):', style=DescriptionStyle(description_width='ini…

Button(description='Run Analysis', disabled=True, layout=Layout(border_bottom='2px solid red', border_left='2p…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [77]:
def dynamicAnalysis():
    # DYNAMIC ground-motion analysis -------------------------------------------------------------
    # create load pattern
    # define acceleration vector from file (dt=0.01 is associated with the input file gm)
    accelSeries  = 900
    ops.timeSeries('Path',accelSeries,'-dt',0.01,'-filePath','BM68elc.acc','-factor',1)     # timeSeries Path accelSeries -dt 0.01 -filePath BM68elc.acc -factor 1;
    ops.pattern('UniformExcitation',2,1,'-accel',accelSeries)     #  define where and how (pattern tag, dof) acceleration is applied
    ops.rayleigh(0.,0.,0.,2*0.02/np.pow(ops.eigen('-fullGenLapack',1)[0],0.5))     #  set damping based on first eigen mode

    # create the analysis
    ops.wipeAnalysis()     #  clear previously-define analysis parameters
    # ops.wipeAnalysis()     # adding this to clear Analysis module 
    ops.constraints('Plain')     #  how it handles boundary conditions
    ops.numberer('Plain')     #  renumber dofs to minimize band-width (optimization), if you want to
    ops.system('BandGeneral')     #  how to store and solve the system of equations in the analysis
    ops.test('NormDispIncr',1.0e-8,10)     #  determine if convergence has been achieved at the end of an iteration step
    ops.algorithm('Newton')     #  use Newtons solution algorithm: updates tangent stiffness at every iteration
    ops.integrator('Newmark',0.5,0.25)     #  determine the next time step for an analysis
    ops.analysis('Transient')     #  define type of analysis: time-dependent
    ops.analyze(1000,0.02)     #  apply 1000 0.02-sec time steps in analysis


    ops.wipe()
    plt.close('all')

    # Load the data
    fname3 = 'Data/DispFreeEx1aEQ.out'
    dataDFree = np.loadtxt(fname3)

    # Create a figure and axes instances
    fig, ax = plt.subplots(1, 1)

    # Plot on the first axis
    ax.set_title('Ex1a. Elastic Cantilever Column EQ Analysis')
    ax.grid(True)
    ax.plot(dataDFree[:, 0], dataDFree[:, 1])
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Free-Node Disp. (m)')

    # Plot on the second axis
    # axes[1].grid(True)
    # axes[1].plot(dataDFree[:, 1], dataDFree[:, 0])
    # axes[1].set_xlabel('Free-Node Disp. (m)')
    # axes[1].set_ylabel('Pseudo-Time (~Force) (s)')

    # Display the plot
    # plt.tight_layout()  # Adjust layout to prevent overlap
    plt.show()

    # End of script
    print('End of Run')


In [78]:
def pushOverAnalysis(disp):

    # define LATERAL load -------------------------------------------------------------
    # Lateral load pattern
    ops.timeSeries('Linear', 2)     # timeSeries Linear 2;
    # define Load Pattern
    ops.pattern('Plain', 2, 2) # 
    Px = 1 # Lateral load
    ops.load(2, Px, 0.0, 0.0)    #  node , FX FY MZ -- REPRESENTATIVE lateral load at top node

    # pushover: diplacement controlled static analysis
    # Note we do not wipe the analysis

    
    print(f'Horizontal displacement at top of column is set to be: {disp} metres')
    steps = 1000 # Must be an integer
    incr = disp / steps
    ops.integrator('DisplacementControl', 2, 1, incr)     #  switch to displacement control, for node 2, dof 1, 0.1 increment
    ops.analyze(steps)


    sfac = opsv.plot_defo(sfac = 1)
    plt.title(f'Deformed shape of column (Scale factor: {sfac})')

    opsv.plot_loads_2d(nep=17, sfac=False, fig_wi_he=False, fig_lbrt=False, 
                   fmt_model_loads={'color': 'black', 'linestyle': 'solid', 'linewidth': 1.2, 'marker': '', 'markersize': 1}, 
                   node_supports=True, truss_node_offset=0, ax=False)
    plt.title('Loads applied to Column (N)')
    plt.xlabel('Metres')
    plt.ylabel('Metres')
    plt.show()
    
    info = ops.printModel()

In [79]:
# output_analysis = widgets.Output(layout={'border': '1px solid black'})

# display(button, output_analysis)

@output_analysis.capture(clear_output = True)
def on_button_clicked(b):
    run_analysis(Py.value, disp.value)
    button.disabled = True

button.on_click(on_button_clicked)


def run_analysis(Py, disp):

    # Define RECORDERS -------------------------------------------------------------

    # https://openseespydoc.readthedocs.io/en/latest/src/nodeRecorder.html#node-recorder-command
    # recorder('Node', '-file', filename, '-time', '-node', *nodeTags=[], '-dof', *dofs=[], respType)
    if analysis.value == 'Push Over':
        name_suffix = 'Push.out'
    else:
        name_suffix = 'EQ.out'

    dofs = [1, 2, 3]
    ops.recorder('Node', '-file', 'Data/DispFreeEx1a' + name_suffix, '-time', '-node', 2, '-dof', *dofs, 'disp')     #  displacements of free node
    ops.recorder('Node', '-file', 'Data/DispBaseEx1a' + name_suffix, '-time', '-node', 1, '-dof', *dofs, 'disp')     #  displacements of support node
    ops.recorder('Node', '-file', 'Data/ReacBaseEx1a' + name_suffix, '-time', '-node', 1, '-dof', *dofs, 'reaction')     #  support reaction

    # https://openseespydoc.readthedocs.io/en/latest/src/elementRecorder.html#element-recorder-command
    # recorder('Element', '-file', filename, '-time', '-ele', *eleTags=[], '-eleRange', startEle, endEle, '-region', regionTag, *args)
    ops.recorder('Element', '-file', 'Data/FColEx1a' + name_suffix, '-time', '-ele', 1, 'globalForce')     #  element forces -- column
    ops.recorder('Element', '-file', 'Data/DColEx1a' + name_suffix, '-time', '-ele', 1, 'deformation')     #  element deformations -- column


    # Define GRAVITY Loads-------------------------------------------------------------
    ops.timeSeries('Linear', 1)  # timeSeries Linear 1;

    # Define Load Pattern
    ops.pattern('Plain', 1, 1)
    ops.load(2, 0.0, Py, 0.0)   #  node , FX FY MZ -- superstructure-weight


    ops.wipeAnalysis()     # adding this to clear Analysis module 
    ops.system('BandGeneral')     #  how to store and solve the system of equations in the analysis
    ops.numberer('Plain')     #  renumber dofs to minimize band-width (optimization), if you want to
    ops.constraints('Plain')     #  how it handles boundary conditions
    ops.integrator('LoadControl',0.1)     #  determine the next time step for an analysis,   apply gravity in 10 steps
    ops.algorithm('Newton')     #  use Newtons solution algorithm: updates tangent stiffness at every iteration
    ops.analysis('Static')     #  define type of analysis static or transient

    ops.test('NormDispIncr',1.0e-8,6)     #  determine if convergence has been achieved at the end of an iteration step
    ops.analyze(10)     #  perform gravity analysis
    ops.loadConst('-time',0.0)     #  hold gravity constant and restart time


    # sfac = opsv.plot_defo(sfac = 100)
    # plt.title('Deformed shape of column after gravity loads have been applied')
    # plt.show()

    if analysis.value == 'Push Over':
        pushOverAnalysis(disp)
    else:
        dynamicAnalysis()

    # # define LATERAL load -------------------------------------------------------------
    # # Lateral load pattern
    # ops.timeSeries('Linear', 2)     # timeSeries Linear 2;
    # # define Load Pattern
    # ops.pattern('Plain', 2, 2) # 
    # Px = 1 # Lateral load
    # ops.load(2, Px, 0.0, 0.0)    #  node , FX FY MZ -- REPRESENTATIVE lateral load at top node

    # # pushover: diplacement controlled static analysis
    # # Note we do not wipe the analysis

    
    # print(f'Horizontal displacement at top of column is set to be: {disp} metres')
    # steps = 1000 # Must be an integer
    # incr = disp / steps
    # ops.integrator('DisplacementControl', 2, 1, incr)     #  switch to displacement control, for node 2, dof 1, 0.1 increment
    # ops.analyze(steps)


    
    
    